## Automated Trading- Buy new predicted stocks over certain %, sell ones that we've had for the duration of maturity period

In [1]:
# access most recently saved directory from the previous step
file_path = "/Users/rileybitterli/Documents/GitHub/SB_VIII_Streamlined/Daily_Workflow/Temp_Files/step_3_to_4.txt"

# read the file and store its contents in a variable
with open(file_path, "r") as file:
    csv_path = file.read()

In [1]:
import pandas as pd
import numpy as np
import robin_stocks.robinhood as rh
import os
import math


In [3]:
last_prediction_batch = pd.read_csv(csv_path)

In [4]:
unique_tickers = last_prediction_batch['ticker'].unique()

In [5]:
test = unique_tickers[:2]

In [6]:
test

array([], dtype=object)

In [7]:
#see if market's open
from datetime import datetime
import pandas_market_calendars as mcal

def is_market_open():
    # Check if today is a weekend
    today = datetime.now()
    if today.weekday() >= 5:  # 0 is Monday, 6 is Sunday
        return False

    # Check if today is a public holiday
    # Using NYSE calendar as an example
    nyse = mcal.get_calendar('NYSE')
    market_open_days = nyse.valid_days(start_date=today, end_date=today)

    return len(market_open_days) > 0

if is_market_open():
    print("Market is open today.")
    # Place your trading script logic here
else:
    print("Market is closed today.")


Market is open today.


In [2]:
import pyotp 
#totp = pyotp.TOTP('[insert your pyotp details]').now()
#rh.login([insert_your_robinhood_email], [insert_your_robinhood_password], mfa_code=totp)

{'access_token': 'eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJkY3QiOjE3MTkzNzE1NTcsImRldmljZV9oYXNoIjoiYmZkNGFjYWRkMjg0MzhmYWFmZjMwOTc2ZWIzNDg1MDQiLCJleHAiOjE3MjUyOTU2MDksImxldmVsMl9hY2Nlc3MiOmZhbHNlLCJtZXRhIjp7Im9pZCI6ImM4MlNIMFdaT3NhYk9YR1Ayc3hxY2ozNEZ4a3ZmbldSWkJLbEJqRlMiLCJvbiI6IlJvYmluaG9vZCJ9LCJvcHRpb25zIjp0cnVlLCJzY29wZSI6ImludGVybmFsIiwic2VydmljZV9yZWNvcmRzIjpbeyJoYWx0ZWQiOmZhbHNlLCJzZXJ2aWNlIjoibnVtbXVzX3VzIiwic2hhcmRfaWQiOjEsInN0YXRlIjoiYXZhaWxhYmxlIn0seyJoYWx0ZWQiOmZhbHNlLCJzZXJ2aWNlIjoiYnJva2ViYWNrX3VzIiwic2hhcmRfaWQiOjQsInN0YXRlIjoiYXZhaWxhYmxlIn1dLCJzbGciOjEsInNscyI6IlBVVnRhRlF1MzdMOTc4V2dSbzBjYW1Gb0p5eE9JM1VHQ2NaYjIrTjE4L0J6OGpKOEpVRS9nc2FNK2xaVER5dEp2a3Q3M0pIek9GLzMycm5xZVBnMUFnPT0iLCJzcm0iOnsiYiI6eyJobCI6ZmFsc2UsInIiOiJ1cyIsInNpZCI6NH0sIm4iOnsiaGwiOmZhbHNlLCJyIjoidXMiLCJzaWQiOjF9fSwidG9rZW4iOiJWeHc1UEl3Z3FaNVdhUW1NTmxnQzdEdXZQVThWVzgiLCJ1c2VyX2lkIjoiYWU3NWQ3ZDctMDg0MC00YTdiLWE5MmItYzNkMWYwNTkxZTM3IiwidXNlcl9vcmlnaW4iOiJVUyJ9.P8lTfuusJqu0b9mzTQZjH6LO14-nbRSBDbchk9u1v0JWNdVf

In [9]:
totp

'858406'

## Get all actively open positions

In [10]:
open_positions = rh.account.get_open_stock_positions()
open_position_tickers = []

# Retrieve ticker symbols for open positions
for position in open_positions:
    instrument_url = position['instrument']
    stock_info = rh.stocks.get_instrument_by_url(instrument_url)
    ticker_symbol = stock_info['symbol']
    open_position_tickers.append(ticker_symbol)

In [11]:
# get details for all open positions

def get_purchase_details_for_position(position):
    instrument_url = position['instrument']
    ticker = rh.stocks.get_symbol_by_url(instrument_url)
    orders = rh.get_all_stock_orders()
    
    purchase_price = None
    purchase_date = None

    for order in orders:
        if order['side'] == 'buy' and order['instrument'] == instrument_url:
            if order['average_price'] is not None:
                purchase_price = float(order['average_price'])
                purchase_date = datetime.fromisoformat(order['created_at']).date()
                break

    # handle the case where purchase_price or purchase_date is not available
    if purchase_price is None or purchase_date is None:
        return {
            'ticker': ticker,
            'purchase_date': np.NaN,
            'purchase_price': np.NaN,
            'quantity': np.NaN,
            'current_profit_or_loss': np.NaN,
        }

    current_price = float(rh.stocks.get_latest_price(ticker)[0])
    quantity = float(position['quantity'])

    profit_or_loss = (current_price - purchase_price) * quantity

    return {
        'ticker': ticker,
        'purchase_date': purchase_date,
        'purchase_price': purchase_price,
        'quantity': quantity,
        'current_profit_or_loss': profit_or_loss
        
    }


# Get purchase details for each open position
purchase_details = [get_purchase_details_for_position(position) for position in open_positions]

current_position_df = pd.DataFrame(purchase_details)

In [12]:
current_position_df

""


## See if we should sell, and if so, add a flag to the dataframe

In [13]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def calculate_sell_criteria(row, days_held=30, loss_threshold=-0.3):
    """
    Calculate the number of days since purchase and determine if a stock should be sold.
    Parameters: 
    param row: A row from the DataFrame.
    param days_held: The number of days to hold before considering selling.
    param loss_threshold: The loss threshold to trigger a sell.
    return: Tuple with number of days since purchase and should_sell flag.
    """
    try:
        purchase_date = datetime.strptime(str(row['purchase_date']), '%Y-%m-%d')
        current_price = float(row['current_profit_or_loss']) / row['quantity'] + row['purchase_price']
        days_since_purchase = (datetime.now() - purchase_date).days

        should_sell = False
        if days_since_purchase >= days_held:
            should_sell = True
        elif (current_price - row['purchase_price']) / row['purchase_price'] <= loss_threshold:
            should_sell = True

        return days_since_purchase, should_sell

    except (TypeError, ValueError):
        return np.NaN, np.NaN

# apply the combined function and create new columns
if not current_position_df.empty:
    current_position_df[['days_since_purchase', 'should_sell']] = current_position_df.apply(
        lambda row: pd.Series(calculate_sell_criteria(row)), axis=1)
    print(current_position_df)
else:
    print("current_position_df is empty, skipping")



current_position_df is empty, skipping


In [14]:
current_position_df

""


# This code sells stock marked with the "should_sell" flag! only run it if you want to close all those positions!

In [15]:
def sell_stock(ticker, quantity):
    """
    sells all available shares of a given stock at market price.

    param ticker: The ticker symbol of the stock to sell.
    param quantity: The number of shares to sell.
    """
    try:
        sell_order = rh.orders.order_sell_market(ticker, quantity)
        return sell_order
    except Exception as e:
        print(f"Error selling {ticker}: {e}")
        return None

# set to keep track of sold tickers and distinctify them
sold_tickers = set()

# execute trades for stocks that should be sold
for index, row in current_position_df.iterrows():
    if row['should_sell']:
        ticker = row['ticker']
        quantity = row['quantity']
        sell_order = sell_stock(ticker, quantity)
        if sell_order is not None:
            print(f"Sold {quantity} shares of {ticker}")
            sold_tickers.add(ticker)

# print the number of different tickers that were sold
print(f"Number of different tickers sold: {len(sold_tickers)}")


Number of different tickers sold: 0


## Return only the tickers that are not actively already involved in an open position

In [16]:

unique_tickers_set = set(unique_tickers)
open_position_tickers_set = set(open_position_tickers)

# tickers in unique_tickers not in open positions
net_new_tickers = list(unique_tickers_set - open_position_tickers_set)

print(net_new_tickers)


[]


## Define Buying Methodology

In [17]:
import math
import smtplib
from email.message import EmailMessage

def buy_stock(ticker, amount):
    try:
        current_price = float(rh.stocks.get_latest_price(ticker)[0])
        shares_to_buy = math.floor(amount / current_price) - 1
        limit_price =  float(round(current_price * 1.07, 2))
        print(ticker)
        print(current_price)
        print(shares_to_buy)

        order = rh.orders.order(symbol =ticker, quantity= shares_to_buy, side = 'buy', limitPrice= limit_price, stopPrice= None, timeInForce='gtc')
        print(f"Order placed for {ticker}: {order}")
        return f"{ticker}: Bought {shares_to_buy} shares at limit price ${limit_price} each"

    except Exception as e:
        print(f"Error placing order for {ticker}: {e}")
        return None

def send_email(subject, content):
    msg = EmailMessage()
    msg.set_content(content)
    msg['Subject'] = subject
    msg['From'] = 'halpitsstockbot@gmail.com'
    msg['To'] = 'riley.bitterli@gmail.com'

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    #server.login([insert your email address], '[insert your password]')
    server.send_message(msg)
    server.quit()

# list to store purchase details
purchase_details = []

# This Actually Executes Trades! Don't run it unless you'd like to buy some stock

In [18]:
for ticker in net_new_tickers:
    result = buy_stock(ticker, 100)
    if result:
        purchase_details.append(result)

# send an email with all purchase details
if purchase_details:
    email_content = "\n".join(purchase_details)
    send_email("Stock Purchase Summary", email_content)


In [19]:
send_email("ran", "step 4 of daily")

## Run to cancel all open orders

In [3]:
#rh.orders.cancel_all_stock_orders()


All Stock Orders Cancelled


[]

## Run to close out all open positions

In [21]:
#open_positions = rh.account.get_open_stock_positions()

    #for position in open_positions:
        #symbol = rh.stocks.get_symbol_by_url(position['instrument'])
        #quantity = float(position['quantity'])

        #if quantity > 0:
            # place a market sell order for all shares
            #try:
                #order = rh.orders.order_sell_market(symbol, quantity)
                #print(f"Market sell order placed for {symbol}: {order}")
            #except Exception as e:
                #print(f"Error placing sell order for {symbol}: {e}")
